In [1]:
import mdtraj as md
import numpy as np
import os
from pyemma import coordinates
# from tqdm import tqdm

In [2]:
phase = 'solvent' # do both solvent and complex (complex is more important)
proj = 13421 # will want to do both complex and solvent
RUNID = 0 # will want to loop over all run IDS

In [3]:
# first copy back pdb file, which contains the input topology needed
os.system(f'scp server@aws3.foldingathome.org:/home/server/server2/projects/{proj}/RUNS/RUN{RUNID}/*pdb .')

0

In [4]:
# load in the topology of the system
topology = md.load(f'hybrid_{phase}.pdb').topology

In [81]:
# now we want the final frame of the trajectory (for the forwards direction) we will want the starting frame also 
# (double check with John which frames correspond to which stages in the cycle)

# frame 0 is start ligand after EQUIL
# 0 -> 1 NEQ
# frame 1 is end ligand after NEQ
# frame 2 is end ligand after EQ
# 1 -> 0 NEQ
# frame 3 is start ligand after NEQ

#for i in {0..99}; do echo $i ; scp server@aws3.foldingathome.org:/home/server/server2/data/SVR314342810/PROJ13420/RUN0/CLONE0/results0/*xtc pos_$i.xtc ; done 
os.system(f'scp server@aws3.foldingathome.org:/home/server/server2/data/SVR314342810/PROJ{proj}/RUN{RUNID}/CLONE0/results0/*xtc pos_0.xtc')
traj = md.load_xtc(f'pos_0.xtc',topology,frame=2)

# for each clone, get the snapshot and append it to the trajectory
# will also want to do this over different GENS in future
n_clones = 100 # will be 100 
n_gens = 1 # will be 6 when everything is done
for gen in range(0,n_gens): 
    for clone in range(1,n_clones):
        print(clone)
        try:
            os.system(f'scp server@aws3.foldingathome.org:/home/server/server2/data/SVR314342810/PROJ{proj}/RUN{RUNID}/CLONE{clone}/results{gen}/*xtc pos_{clone}.xtc')
            new_frame = md.load_xtc(f'pos_{clone}.xtc',topology,frame=3)
            traj = traj.join(new_frame)
        except ValueError:
            print(f'error for clone {clone} gen {gen}')
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
error for clone 15 gen 0
16
17
18
error for clone 18 gen 0
19
20
21
22
23
24
25
26
27
28
error for clone 28 gen 0
29
error for clone 29 gen 0
30
error for clone 30 gen 0
31
32
33
error for clone 33 gen 0
34
35
error for clone 35 gen 0
36
37
error for clone 37 gen 0
38
error for clone 38 gen 0
39
error for clone 39 gen 0
40
error for clone 40 gen 0
41
42
43
error for clone 43 gen 0
44
error for clone 44 gen 0
45
error for clone 45 gen 0
46
error for clone 46 gen 0
47
error for clone 47 gen 0
48
error for clone 48 gen 0
49
error for clone 49 gen 0
50
error for clone 50 gen 0
51
error for clone 51 gen 0
52
error for clone 52 gen 0
53
error for clone 53 gen 0
54
error for clone 54 gen 0
55
error for clone 55 gen 0
56
error for clone 56 gen 0
57
error for clone 57 gen 0
58
error for clone 58 gen 0
59
error for clone 59 gen 0
60
error for clone 60 gen 0
61
error for clone 61 gen 0
62
error for clone 62 gen 0
63
error for clone 63 gen 0
64
error for clone 6

In [82]:
print(traj.n_frames) # this should match the number of (clones x gens)

32


In [83]:
# find the atom indices for the ligand (resname 'MOL')
mol_ids = []
for res in topology.residues:
    if res.name == 'MOL':
        for atom in res.atoms:
            mol_ids.append(atom.index)

In [84]:
# align the frames
traj.superpose(traj,atom_indices=mol_ids)
#only keep the ligand 
traj.atom_slice(mol_ids, inplace=True)
traj.save_pdb('testing.pdb')

In [85]:
###
# now do the clustering
# find 'centroid' of each clusters
# additionally --- could 'weigh' configurations by work values, but I think we would want frame 1 for this? not sure

In [166]:
###
# now do the clustering
# find 'centroid' of each clusters
# additionally --- could 'weigh' configurations by work values, but I think we would want frame 1 for this? not sure

n_k_means = 5 # not sure what is a good number for this

clust_alg = coordinates.cluster_kmeans(k=n_k_means)

# flatten the trajectory into a 2D array T x (3n) array, rather than T x n x 3 
i,j,k = np.shape(traj.xyz)
xyz_flat = np.reshape(traj.xyz, (i,j*k)) # we probably want to drop hydrogens from this clustering

# do the clustering
clust_alg.assign(xyz_flat)

representative_frame = {i:None for i in frame_to_clust.values()}

for f,c in enumerate(clust_alg.get_output()[0].flatten()):
    coords = test[f]
    clust_center = clust_alg.clustercenters[c]
    dist = np.linalg.norm(coords-clust_center)
    if representative_frame[c] is None or dist < representative_frame[c][1]:
        representative_frame[c] = (f, dist) # chose as representative frame if there isn't one yet, or if it's the closest
print(representative_frame)

# pull out the representative frames
clusters = traj[[i[0] for i in representative_frame.values()]]

### saving as an sdf file would be better.
clusters.save_pdb(f'RUN{RUNID}-{phase}.pdb')

{2: None, 3: None, 0: None, 1: None, 4: None}
{2: (9, 0.7974571), 3: (4, 0.7834148), 0: (2, 0.43813148), 1: (29, 0.7402873), 4: (17, 0.6342507)}
